In [29]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


### Loading a CSV file into DATAFRAME

In [30]:
data1 = pd.read_excel('C:/Users/Admin/Downloads/output_all_students_Train_v10.xlsx')
data = data1.copy()

### שמנו פה את כל הנתונים שיצרנו לתוך פונקציה כדי לעשות ניבוי למחיר

In [31]:
def prepare_data(data):
    import pandas as pd
    import numpy as np
    import re
    import matplotlib.pyplot as plt 

    data_filter = data.dropna(subset=["price"])

    data_filter['City'] = data_filter['City'].str.replace('נהרייה', 'נהריה')
    data_filter['City'] = data_filter['City'].str.replace(' שוהם', 'שוהם')
    for col in ['City', 'type','Street' ,'condition ']:
        if data_filter[col].dtype == 'object':
            data_filter[col] = data_filter[col].str.strip()

    def extract_number(price):
        if isinstance(price, str):
            # Remove any non-digit characters using regular expressions
            numeric_part = re.sub(r'[^0-9]', '', price)
            try:
                # Convert the remaining string to an integer
                number = int(numeric_part)
                return number
            except ValueError:
                return np.nan
        else:
            return price

    data_filter.loc[:, 'price'] = data_filter['price'].apply(extract_number)
    data_filter.loc[:, 'Area'] = data_filter['Area'].apply(extract_number)


    def only_digits(value):
        if isinstance(value, float):
            return value
        elif isinstance(value, str):
            numbers = ''.join(filter(lambda x: x.isdigit() or x == '.', value))
            if numbers:
                return float(numbers)
        return value
    data_filter['room_number'] = data_filter['room_number'].apply(only_digits)

    data_filter.loc[:, 'Street'] = data_filter["Street"].apply(lambda x: re.sub(r'[^א-ת\s]', '', str(x)))
    data_filter.loc[:, 'city_area'] = data_filter["city_area"].apply(lambda x: re.sub(r'[^א-ת\s]', '', str(x)))
    data_filter.loc[:, 'description '] = data_filter["description "].apply(lambda x: re.sub(r'[^א-ת0-9\s]', '', str(x)))

    mapping = {
        'שמור': 1, 'חדש': 1, 'משופץ': 1, 'חדש מקבלן': 1, 'מצב הנכס:משופץ': 1, 'מצב הנכס:שמור': 1,
        'מצב הנכס:חדש': 1, 'מצב הנכס:חדש מקבלן': 1, 'Save': 1, 'new': 1, 'renovated': 1, 'כן': 1,
        'יש מחסן': 1, 'יש סורגים': 1, 'יש חניה': 1, 'יש חנייה': 1, 'יש מעלית': 1, 'יש מיזוג אוויר': 1,'יש מיזוג אויר': 1, 'יש מרפסת': 1,
        'נגיש לנכים': 1, 'יש': 1,'אין':0, 'נגיש': 1, 'יש ממ"ד': 1,'יש ממ״ד': 1, 'יש מיזוג אויר': 1, 'yes': 1,
        'לא צויין': 0, 'מצב הנכס:לא צויין': 0, 'ישן': 0, 'לא': 0, 'אין ממ"ד': 0,'דורש שיפוץ': 0, 'לא נגיש לנכים': 0,
        'אין ממ״ד': 0, 'אין מרפסת': 0, 'אין מחסן': 0, 'אין סורגים': 0, 'אין חניה': 0, 'אין מעלית': 0,
        'אין מיזוג אויר': 0, 'no': 0, 'לא נגיש': 0,
        True: 1, False: 0  # Mapping True to 1 and False to 0
    }
    columns_to_map = ['hasElevator ', 'hasParking ', 'hasBars ','hasStorage ','hasAirCondition ','hasBalcony ','hasMamad ','handicapFriendly ']  # List of columns to map
    data_filter[columns_to_map] = data_filter[columns_to_map].replace(mapping)

    import re
    import numpy as np

    lines = data_filter["floor_out_of"].fillna('').values

    new_column = ['floor']
    total_floors_column = []

    for line in lines:
        match = re.findall(r'\d+', str(line))
        if len(match) >= 2:
            new_column.append(match[0])
            total_floors_column.append(match[1])
        else:
            new_column.append(np.nan)
            total_floors_column.append(np.nan)

    # Check for length mismatch
    if len(new_column) > len(data_filter):
        new_column = new_column[:len(data_filter)]
    if len(total_floors_column) > len(data_filter):
        total_floors_column = total_floors_column[:len(data_filter)]

    # Add the new columns to the DataFrame
    data_filter['floor'] = new_column
    data_filter['total_floors'] = total_floors_column

    import numpy as np

    lines = data_filter["floor_out_of"].fillna('').values

    new_column = []

    for line in lines:
        if "קומת קרקע" in line:
            new_column.append('0')
        else:
            match = re.search(r'\d+', str(line))
            if match:
                new_column.append(match.group())
            else:
                new_column.append(np.nan)

    # Adjust the length of new_column to match the DataFrame index
    missing_entries = len(data_filter) - len(new_column)
    new_column += [np.nan] * missing_entries

    # Change the first element in the new_column with the first number from "floor_out_of" column
    first_floor_out_of = data_filter["floor_out_of"].fillna('').values[0]
    match = re.search(r'\d+', str(first_floor_out_of))
    if match:
        new_column[0] = match.group()

    # Add the new column to the DataFrame
    data_filter['floor'] = new_column

    data_filter['entranceDate '] = data_filter['entranceDate '].replace('גמיש', 'flexible')
    data_filter['entranceDate '] = data_filter['entranceDate '].replace('לא צויין', 'not_defined')
    data_filter['entranceDate '] = data_filter['entranceDate '].replace('מיידי', 'Now')

    valid_values = ['flexible', 'not_defined', 'Now']

    conversion_dict = {value: pd.to_datetime(value, errors='coerce') for value in valid_values}

    data_filter['entranceDate '] = data_filter['entranceDate '].map(conversion_dict).fillna(data_filter['entranceDate '])

    # Retrieve the current date
    current_date = pd.to_datetime('today').normalize()


    # Iterate through each row
    for index, row in data_filter.iterrows():
        if pd.isnull(row['entranceDate ']) or pd.isna(row['entranceDate ']):
            continue  # Skip rows with missing or NaT values
        elif isinstance(row['entranceDate '], str):
            continue  # Skip rows with string values
        else:
            # Calculate the duration in months
            duration = (row['entranceDate '].year - current_date.year) * 12 + (row['entranceDate '].month - current_date.month)

            # Assign category based on duration
            if duration < 6:
                category = 'less_than_6 months'
            elif 6 <= duration <= 12:
                category = 'months_6_12'
            else:
                category = 'above_year'

        # Update the row with the assigned category
        data_filter.at[index, 'entranceDate '] = category

    import pandas as pd

    # Assuming your DataFrame is called df and the city column is named 'city'
    cities = data_filter['City']

    # Assigning unique numbers to each city
    city_codes, unique_cities = pd.factorize(cities)

    # Creating a new column with the city codes
    data_filter['city_code'] = city_codes

    # Printing the updated DataFrame

    ## 1-Central
    city_to_region = {
        'פתח תקווה': 'Central',
        'נתניה': 'Central',
        'באר שבע': 'Southern',
        'הרצליה': 'Central',
        'אריאל': 'Judea and Samaria',
        'דימונה': 'Southern',
        'רחובות': 'Central',
        'גבעת שמואל': 'Central',
        'ירושלים': 'Jerusalem',
        'שוהם': 'Central',
        'כפר סבא': 'Central',
        'רעננה': 'Central',
        'נהריה': 'Northern',
        'זכרון יעקב': 'Central',
        'קרית ביאליק': 'Northern',
        'חיפה': 'Northern',
        'הוד השרון': 'Central',
        'תל אביב': 'Tel Aviv',
        'ראשון לציון': 'Central',
        'יהוד מונוסון': 'Central',
        'נס ציונה': 'Central',
        'אילת': 'Southern',
        'חולון': 'Central',
        'מודיעין מכבים רעות': 'Central',
        'צפת': 'Northern',
        'בת ים': 'Tel Aviv',
        'רמת גן': 'Tel Aviv',
        'נוף הגליל': 'Northern',
        'בית שאן': 'Northern'
    }
    region_to_number = {
        'פתח תקווה': 1,
        'נתניה': 1,
        'באר שבע': 2,
        'הרצליה': 1,
        'אריאל': 3,
        'דימונה': 2,
        'רחובות': 1,
        'גבעת שמואל': 1,
        'ירושלים': 4,
        'שוהם': 1,
        'כפר סבא': 1,
        'רעננה': 1,
        'נהריה': 5,
        'זכרון יעקב': 1,
        'קרית ביאליק': 5,
        'חיפה': 5,
        'הוד השרון': 1,
        'תל אביב': 6,
        'ראשון לציון': 1,
        'יהוד מונוסון': 1,
        'נס ציונה': 1,
        'אילת': 2,
        'חולון': 1,
        'מודיעין מכבים רעות': 1,
        'צפת': 5,
        'בת ים': 6,
        'רמת גן': 6,
        'נוף הגליל': 5,
        'בית שאן': 5
    }

    data_filter['Region'] = data_filter['City'].map(city_to_region)
    data_filter['Region_number'] = data_filter['City'].map(region_to_number)

    import pandas as pd

    # Assuming your DataFrame is called df and the city column is named 'city'
    types = data_filter['type']

    # Assigning unique numbers to each city
    city_codes, unique_cities = pd.factorize(types)

    # Creating a new column with the city codes
    data_filter['type_num'] = city_codes

    # Printing the updated DataFrame

    selected_columns = ['Region_number','type_num', 'room_number', 'Area', 'price', 'floor', 'hasElevator ', 'hasParking ' , 'hasBars ','hasStorage ','condition ',
                        'hasAirCondition ','hasBalcony ', 'hasMamad ', 'handicapFriendly ', 'furniture ']

    # Create a new DataFrame using the selected columns from the original DataFrame
    new_df = data_filter[selected_columns].copy()
    new_df = new_df.dropna(subset=["price"])

    # Print the new DataFrame
    new_df.head()


    # Assuming your DataFrame is named 'df'
    new_df['condition '] = new_df['condition '].map({'שמור': 1, 'חדש': 1, 'משופץ': 1, 'לא צויין': 0, 'ישן': 0, 'דורש שיפוץ': 0})
    new_df['furniture '] = new_df['furniture '].map({'חלקי': 1, 'מלא': 1, 'לא צויין': 0, 'אין': 0})


    new_df['room_number'] = pd.to_numeric(new_df['room_number'], errors='coerce')

    # Changing other columns to float type
    new_df['room_number'] = pd.to_numeric(new_df['room_number'], errors='coerce')

    # Changing other columns to float type
    float_columns = ['Region_number','condition ','furniture ',
                     'type_num', 'room_number', 'Area', 'price', 'floor',
                     'hasElevator ', 'hasParking ', 'hasBars ', 'hasStorage ', 'hasAirCondition ',
                     'hasBalcony ', 'hasMamad ', 'handicapFriendly ']

    new_df[float_columns] = new_df[float_columns].astype(float)

    new_df.loc[new_df["Area"] > 800, "Area"] /= 10
    new_df.loc[new_df["room_number"] > 20.0, "room_number"] /= 10

    columns_to_fill = ['hasElevator ', 'hasParking ', 'hasBars ', 'hasStorage ','condition ','furniture ',
                        'hasAirCondition ', 'hasBalcony ', 'hasMamad ',
                       'handicapFriendly ']

    new_df[columns_to_fill] = new_df[columns_to_fill].fillna(0)
    from sklearn.impute import KNNImputer

    new_df = pd.get_dummies(new_df)

    # Initialize the KNNImputer
    imputer = KNNImputer(n_neighbors=5)  # You can adjust the number of neighbors as needed

    # Perform the imputation
    new_df = pd.DataFrame(imputer.fit_transform(new_df), columns=new_df.columns)
    
    return new_df

    

## הפכנו את כל העמודות לצורה מספרית - חלק בינארי וחלק לפי אינדקס

In [32]:
import pandas as pd

# Assuming you have your data in a DataFrame called 'data'
processed_data = prepare_data(data)

# You can then use the processed_data DataFrame for further analysis or operations
# For example, you can print the processed_data DataFrame
processed_data.head()


C:\Users\Admin\AppData\Local\Temp\ipykernel_22496\2778307729.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filter['City'] = data_filter['City'].str.replace('נהרייה', 'נהריה')
C:\Users\Admin\AppData\Local\Temp\ipykernel_22496\2778307729.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filter['City'] = data_filter['City'].str.replace(' שוהם', 'שוהם')
C:\Users\Admin\AppData\Local\Temp\ipykernel_22496\2778307729.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,Region_number,type_num,room_number,Area,price,floor,hasElevator,hasParking,hasBars,hasStorage,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,furniture
0,1.0,0.0,5.5,137.0,3600000.0,11.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0
1,1.0,0.0,3.0,84.0,2550000.0,6.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,4.0,120.0,2650000.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
3,1.0,0.0,3.5,110.0,2450000.0,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,1.0,0.0,4.5,120.0,2720000.0,3.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0


#### From Here you can drop the Price and check the model
Region_numbers -
        'פתח תקווה': 1,
       'נתניה': 1,
        'באר שבע': 2,
        'הרצליה': 1,
        'אריאל': 3,
        'דימונה': 2,
        'רחובות': 1,
        'גבעת שמואל': 1,
        'ירושלים': 4,
        'שוהם': 1,
        'כפר סבא': 1,
        'רעננה': 1,
        'נהריה': 5,
        'זכרון יעקב': 1,
        'קרית ביאליק': 5,
        'חיפה': 5,
       'הוד השרון': 1,
        'תל אביב': 6,
        'ראשון לציון': 1,
        'יהוד מונוסון': 1,
       'נס ציונה': 1,
       'אילת': 2,
       'חולון': 1,
        'מודיעין מכבים רעות': 1,
       'צפת': 5,
        'בת ים': 6,
        'רמת גן': 6,
        'נוף הגליל': 5,
        'בית שאן': 5


In [33]:
processed_data.isna().sum().sum()

0

In [34]:
processed_data.describe()

,Region_number,type_num,room_number,Area,price,floor,hasElevator,hasParking,hasBars,hasStorage,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,furniture
count,693.000000,693.000000,693.000000,693.000000,6.930000e+02,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000
mean,2.370851,1.373737,4.313997,122.489177,3.840591e+06,3.195382,0.650794,0.518038,0.347763,0.458874,0.923521,0.832612,0.536797,0.580087,0.470418,0.246753
std,1.889158,3.088343,1.228923,57.357853,1.500778e+07,3.607856,0.477064,0.500035,0.476604,0.498666,0.265955,0.373592,0.499004,0.493901,0.499485,0.431433
min,1.000000,0.000000,1.000000,30.000000,1.000000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,3.500000,88.000000,2.120000e+06,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,4.000000,110.000000,2.900000e+06,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
75%,4.000000,1.000000,5.000000,138.000000,3.880000e+06,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,6.000000,15.000000,10.000000,504.000000,3.950000e+08,29.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [35]:
X = processed_data.drop('price', axis=1)  # Assuming 'price' is the column name for the target variable
y = processed_data['price']

In [36]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [38]:
##model = ElasticNet()
model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=10, random_state=42)
model.fit(X_train, y_train)


ElasticNetCV(cv=10, l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
             random_state=42)

In [39]:
y_pred = model.predict(X_test)
print("R^2 Score: ", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error: ", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


R^2 Score:  -0.12691928060597113
Mean Absolute Error:  1591487.8684606196
Mean Squared Error:  4158276718265.099
Root Mean Squared Error:  2039185.3074855898


In [40]:
print('Optimal alpha: ', model.alpha_)
print('Optimal l1 ratio: ', model.l1_ratio_)


Optimal alpha:  2440.6446384948026
Optimal l1 ratio:  0.99


In [41]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 4158276718265.099


In [42]:
pickle.dump(model, open("model.pkl","wb"))

In [43]:
filename1 = "Dataset_for_test.xlsx"
path = 'C:/Users/Admin/Downloads/'
datafile1 =  path + filename1
test = pd.read_excel(datafile1, header=0)
test = process_data(test)
X_test = test.drop('price', axis=1)
y_test = test['price']

In [44]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('MSE with best parameters:', len(str(int(mse))) )
print('MSE with best parameters:',mse**0.5 )

MSE with best parameters: 17
MSE with best parameters: 143803128.74955568


C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but ElasticNetCV was fitted without feature names
  warnings.warn(
